In [ ]:
#required modules
import numpy as np
import pandas as pd
import os
import random
from matplotlib import pyplot as plt
from  sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from datetime import datetime
from datetime import timedelta

#customized moduless filt
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.core.Datasets import Datasets
from ptrail.features.kinematic_features import KinematicFeatures as kinematic
from ptrail.preprocessing.filters import Filters as filt

In [ ]:
!cp drive/MyDrive/project_codes/DATASETS/train.csv .

In [ ]:
df=pd.read_csv('train.csv')
df.head()

In [ ]:
len(df)

In [ ]:
start=0
end=400000
for j in range(start,end,100000):
  flag=False

  for i in range(j,j+100000):

      trip=df.iloc[i]
      #list of longitude and latiude  at original time stamp
      trip_location=trip['POLYLINE']
      value =eval(trip_location.split()[0])
      base_time=datetime.fromtimestamp(trip['TIMESTAMP'])
      time_list=[base_time+ timedelta(seconds=j*15) for j in range(len(value))]
      tmp_dataframe=pd.DataFrame(value,columns=['longitude','latitude'])

      taxi_id_list=[trip['TAXI_ID'] for k in range(len(value))]
      tmp_dataframe['TIMEDATA']=time_list
      tmp_dataframe['TAXI_ID']=taxi_id_list
      tmp_dataframe=PTRAILDataFrame(data_set=tmp_dataframe,latitude='latitude',longitude='longitude',datetime='TIMEDATA',traj_id='TAXI_ID')
      tmp_dataframe=kinematic.create_acceleration_column(tmp_dataframe)
      tmp_dataframe=kinematic.create_bearing_column(tmp_dataframe)
      if flag==False:
        tmp_dataframe.to_csv('data_with_feature/kaggle_data_with_feature_{}.csv'.format(j),index=False,header=True)
        flag=True
      else:
        tmp_dataframe.to_csv('data_with_feature/kaggle_data_with_feature_{}.csv'.format(j),mode='a',index=False,header=False)



In [ ]:
!cp new_data.csv drive/MyDrive/

In [ ]:
min_longitude=-8.53901
max_longitude=-8.53848
min_latitude=41.14460
max_latitude=41.14508
intersection_points = df.loc[(df['longitude']<max_longitude)&
                                            (df['longitude']>min_longitude)&
                                            (df['latitude']<max_latitude)&
                                            (df['latitude']>min_latitude)]

print("lenght of data is {}".format(len(intersection_points)))
samples=int(input('ENTER NUMBER OF SAMPLES TO SHOW : '))
samples_points=intersection_points.sample(samples)
plt.scatter(samples_points.longitude,samples_points.latitude)
plt.show()
print('############################################')
save=input('DO YOU WANT TO SAVE YES,NO ? : ')
if save=='yes':
  intersection=pd.DataFrame([len(intersection_points)],columns=['lenght_of_data_at_intersection'])
  intersection['min_longitude']=min_longitude
  intersection['max_longitude']=max_longitude
  intersection['min_latitude']=min_latitude
  intersection['max_latitude']=max_latitude
  intersection.to_csv('points.csv',mode='a',header=False)


In [ ]:
data_mining

In [ ]:
!cp points.csv drive/MyDrive/project_codes/DATASETS/

In [ ]:
c=pd.read_csv('points.csv')
c.info()